In [26]:
from collections import defaultdict
import pandas as pd
from surprise import Reader, Dataset
from surprise import KNNWithMeans, KNNBasic, SVD
from surprise import accuracy
from surprise.model_selection import train_test_split, cross_validate

import matplotlib.pyplot as plt
import numpy as np
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
class RefinedMyAlgo():
    def __init__(self, rating_data='', data_frame='', movie_data=''):
        if rating_data:
            reader = Reader(line_format='user item rating timestamp', sep=',')
            self.ratings = Dataset.load_from_file(rating_data, reader)
#             self.trainset, self.testset = train_test_split(self.ratings, test_size=0.25)
            self.trainset = self.ratings.build_full_trainset()
            self.sim_options = {'name': 'cosine','user_based': False}
        elif not data_frame.empty:
            reader = Reader(rating_scale=(0, 5))
            self.ratings = Dataset.load_from_df(data_frame[['userId', 'movieId', 'rating']], reader)
            self.trainset = self.ratings.build_full_trainset()
            self.sim_options = {'name': 'cosine','user_based': False}
            
        if movie_data:
            self.movies = pd.read_csv(movie_data, low_memory=False)
            self.movies['year'] = self.movies['title'].apply(lambda x: x[-5:-1])
            self.movies['title'] = self.movies['title'].apply(lambda x: x[:-7])
            self.movies['genres'] = self.movies['genres'].apply(lambda x: x.replace('|',', '))

        
    def set_k(self, k_value=''):
        if k_value:
            algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
            self.algo = algo
            self.algo.fit(self.trainset)
        else:
            algo = SVD()
            self.algo = algo
            self.algo.fit(self.trainset)
        
        
    def find_best_k(self, k_value=''):
        if k_value:
            print('K = {}'.format(k_value))
            algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
            return cross_validate(algo, self.ratings, measures=['RMSE', 'MAE'], cv=10, verbose=True)
        else:
            aux = []
            for k_value in [3, 5, 7, 10, 15, 20, 30, 40]:
                print('K = {}'.format(k_value))
                algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
                my_dict = cross_validate(algo, self.ratings, measures=['RMSE', 'MAE'], cv=10, verbose=False)
                my_dict['k_value'] = k_value
                aux.append(my_dict)
            return aux
    
    
    def set_testset(self, users):
        if users:
            user_ratings = self.trainset.ur
            movies_ids = list(self.movies['movieId'])
            global_mean=self.trainset.global_mean
            my_testset = []
            
            for user in users:
                iuid = self.trainset.to_inner_uid(str(user))
                for movie in movies_ids:
                    is_in = False
                    for rating in user_ratings[iuid]:
#                         print( 'MOVIE: {}, RATING: {}'.format(movie,bla.trainset.to_raw_iid(rating[0])) )
                        if int(movie) == int(self.trainset.to_raw_iid(int(rating[0]))):
                            is_in = True
                            break
                    if not is_in:
                        my_tuple = (str(user),str(movie),global_mean)
                        my_testset.append(my_tuple)
                        
            self.testset = my_testset
        else:
            testset = self.trainset.build_anti_testset()
            self.testset = testset
        return self.testset


    def predict_ratings(self,users=''):
        # # Predict ratings for all pairs (u, i) that are NOT in the training set.
#         testset = self.trainset.build_anti_testset()
#         self.testset = testset
        testset = self.set_testset(users)
        predictions = self.algo.test(testset)
        self.predictions = predictions
        
        
    def set_perfil_movies(self, users):
        metadata = pd.read_csv('ml-latest-small/ratings.csv', low_memory=False, names=['userId', 'movieId', 'rating','timestamp'])
        metadata = metadata.drop(columns="timestamp")

        metadata_filtered = metadata[metadata.userId.isin(users)]

        self.group_sparse_mtx = pd.pivot_table(metadata_filtered, values='rating', index=['userId'], columns=['movieId'], fill_value=0)
        
        self.perfil_movies = list(self.group_sparse_mtx)
        
    
    ### You must call self.set_perfil_movies() before
    def set_candidate_movies(self):
        candidate_movies = []
        for item in refinedMyAlgo.movies.iterrows():
        #     get the movieId of each movie in movies dataframe
            if item[1].values[0] not in self.perfil_movies:
                candidate_movies.append(item[1].values[0])
        self.candidate_movies = candidate_movies
        
        
    def calc_similarity_matrix(self):
        #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
        tfidf = TfidfVectorizer(stop_words='english')
        
        #Replace NaN with an empty string
        self.movies['title'] = self.movies['title'].fillna('')
        self.movies['genres'] = self.movies['genres'].fillna('')
        
        #Construct the required TF-IDF matrix by fitting and transforming the data
        tfidf_matrix_title = tfidf.fit_transform(self.movies['title'])
        tfidf_matrix_genres = tfidf.fit_transform(self.movies['genres'])
        
        #Compute the cosine similarity matrix
        self.cosine_sim_movies_title = cosine_similarity(tfidf_matrix_title, tfidf_matrix_title)
        self.cosine_sim_movies_genres = cosine_similarity(tfidf_matrix_genres, tfidf_matrix_genres)
        
        
    def get_similar_movies(self, references, title_weight=0.5):
        recs = []
        for movie in references:
            # Get the pairwsie similarity scores of all movies with that movie
            movie_idx = int(self.movies[self.movies['movieId']==movie['movieID']].index[0])
            sim_scores_title = list(enumerate(self.cosine_sim_movies_title[movie_idx]))
            sim_scores_genres = list(enumerate(self.cosine_sim_movies_genres[movie_idx]))
            
            # Calculate total similarity based on title and genres
            total_sim_score = []
            for i in range(len(sim_scores_title)):
                aux = (sim_scores_title[i][1]*title_weight) + (sim_scores_genres[i][1]*(1-title_weight))
                total_sim_score.append((i, aux))
                
            # Sort the movies based on the similarity scores
            total_sim_score = sorted(total_sim_score, key=lambda x: x[1], reverse=True)
            
            candidates_sim_score = []
            for item in total_sim_score:
                if self.movies.loc[item[0]].values[0] not in self.perfil_movies:
                    candidates_sim_score.append(item)
            
            # Get the scores of the 10 most similar movies
            candidates_sim_score = candidates_sim_score[1:11]
            
            recs.append(candidates_sim_score)
            
        return recs
    
    
    def get_relevance_score(self, recs, references):
        count = 0
        recs_dict = []
        for reference in references:
        #     print('Referência: {}\t gêneros: {}'.format(refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference['movieID']].values[0][1], refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference['movieID']].values[0][2]))

            for movie in recs[count]:
                aux = {}

                movie_id = self.movies.loc[movie[0]].values[0]
                movie_title = self.movies.loc[movie[0]].values[1]
                movie_genres = self.movies.loc[movie[0]].values[2]
                movie_similarity = movie[1]
                movie_relevance = ( reference['rating']/5.0 )*movie_similarity

                aux['movie_id'] = movie_id
                aux['movie_title'] = movie_title
                aux['movie_genres'] = movie_genres
                aux['movie_similarity'] = movie_similarity
                aux['movie_relevance'] = movie_relevance

                recs_dict.append(aux)

        #         print('\tSim: {},\trelevance: {},\tmovieId: {},\ttitle: {}'.format(aux['movie_similarity'], aux['movie_relevance'], aux['movie_id'], aux['movie_title']))

            count=count+1

        recs_dict = sorted(recs_dict, key = lambda i: i['movie_relevance'],reverse=True)

        return recs_dict
    
    
    def calc_distance_item_in_list(self, item, this_list, title_weight=0.5):

        idx_i = int(self.movies[self.movies['movieId']==item['movie_id']].index[0])

        total_dist = 0
        for movie in list_r:
            idx_j = int(self.movies[self.movies['movieId']==movie['movie_id']].index[0])

            sim_i_j = (self.cosine_sim_movies_title[idx_i][idx_j]*title_weight) + (self.cosine_sim_movies_genres[idx_i][idx_j]*(1-title_weight))
            dist_i_j = 1 - sim_i_j
            total_dist = total_dist + dist_i_j

        result = total_dist/len(list_r)
        return result

In [28]:
refinedMyAlgo = RefinedMyAlgo(rating_data='ml-latest-small/ratings.csv', movie_data='ml-latest-small/movies.csv')
refinedMyAlgo.set_k()

In [29]:
my_users = [77,596,452,243,420]

refinedMyAlgo.predict_ratings(users=my_users)
len(refinedMyAlgo.predictions)

47891

In [30]:
# metadata = pd.read_csv('ml-latest-small/ratings.csv', low_memory=False, names=['userId', 'movieId', 'rating','timestamp'])
# metadata = metadata.drop(columns="timestamp")

# metadata_filtered = metadata[metadata.userId.isin(my_users)]

# my_group_sparse = pd.pivot_table(metadata_filtered, values='rating', index=['userId'], columns=['movieId'], fill_value=0)
# my_group_sparse.head()

refinedMyAlgo.set_perfil_movies(users=my_users)
refinedMyAlgo.set_candidate_movies()

# print(refinedMyAlgo.perfil_movies)
# print(refinedMyAlgo.candidate_movies)
refinedMyAlgo.group_sparse_mtx.head()

movieId,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
userId,,,,,,,,,,,,,,,,,,,,,
77,0,0,0.0,0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
243,0,5,0.0,0,4,0,4,0,4,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
420,4,0,3.5,0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
452,0,4,0.0,0,0,0,4,5,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
596,4,0,3.5,4,0,4,0,0,0,3.5,...,2.5,3.5,3.5,4,3.5,3.5,3.5,3.5,4,4


In [31]:
group_filled_mtx = refinedMyAlgo.group_sparse_mtx.copy()

for index, row in group_filled_mtx.iterrows():
    for col in list(group_filled_mtx):
        if(group_filled_mtx.loc[index,col] == 0.0):
            aux = list(filter(lambda x: x.uid==str(index) and x.iid==str(col), refinedMyAlgo.predictions))
            group_filled_mtx.loc[index,col] = aux[0].est

group_filled_mtx.head()

movieId,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
userId,,,,,,,,,,,,,,,,,,,,,
77,3.911011,3.806462,3.750705,3.534394,3.746047,3.417996,2.672040,3.868550,3.077827,4.245173,...,3.303467,3.264086,3.657607,3.574377,3.626423,3.592790,3.535861,3.470167,3.626137,3.924926
243,4.550381,5.000000,4.631711,4.475887,4.000000,4.064885,4.000000,4.520687,4.000000,4.881109,...,3.876472,3.899536,4.217576,4.307999,3.890789,4.259610,4.185807,4.072341,4.165670,4.542074
420,4.000000,3.821810,3.500000,3.660236,3.576090,3.319272,2.679261,4.109293,3.463963,4.078787,...,3.340176,3.325728,3.567521,3.849116,3.412003,3.642478,3.635395,3.529855,3.683991,3.692343
452,4.830452,4.000000,4.911582,4.617298,4.567072,4.281766,4.000000,5.000000,4.294977,4.998342,...,4.163771,4.184727,4.347793,4.612599,4.315139,4.363743,4.564877,4.376564,4.670424,4.563446
596,4.000000,3.354786,3.500000,4.000000,3.739391,4.000000,2.782331,3.846402,3.112233,3.500000,...,2.500000,3.500000,3.500000,4.000000,3.500000,3.500000,3.500000,3.500000,4.000000,4.000000


In [32]:
########################################################################
# # Implementing least misery ending-up in a dataframe
########################################################################
values = []
labels = []
for i in range(0,len(list(group_filled_mtx))):
    my_col = group_filled_mtx.iloc[ : ,i]
    label = my_col.name
    my_col = list(my_col)
    
    labels.append(label)
    values.append( float(min(my_col)) )
    
# print('Array values: {}, Array labels: {}'.format(values, labels))
agg_group_perf = pd.DataFrame(index=[900], columns=labels)

for i in range(0,len(list(agg_group_perf))):
    agg_group_perf.iloc[0, i] = values[i]

agg_group_perf.head()

,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
900,3.91101,3.35479,3.5,3.53439,3.57609,3.31927,2.67204,3.8464,3.07783,3.5,...,2.5,3.26409,3.5,3.57438,3.412,3.5,3.5,3.47017,3.62614,3.69234


In [33]:
group_pref_dict = []
for col in list(agg_group_perf):
    my_dict = {}
#     print('Valor: {}, Coluna: {}'.format(agg_group_perf.loc[900,col], col))
    my_dict['rating'] = agg_group_perf.loc[900,col]
    my_dict['movieID'] = col
    group_pref_dict.append(my_dict)
    
group_pref_dict = sorted(group_pref_dict, key = lambda i: i['rating'],reverse=True)
group_pref_dict

[{'rating': 4.33073344799974, 'movieID': 318},
 {'rating': 4.2775426193631025, 'movieID': 904},
 {'rating': 4.235059436881213, 'movieID': 6350},
 {'rating': 4.230236874437601, 'movieID': 5618},
 {'rating': 4.216759592604618, 'movieID': 38061},
 {'rating': 4.194045376892017, 'movieID': 60069},
 {'rating': 4.130634843518455, 'movieID': 31658},
 {'rating': 4.100544973206452, 'movieID': 3000},
 {'rating': 4.076856035267363, 'movieID': 4011},
 {'rating': 4.072612700965247, 'movieID': 2160},
 {'rating': 4.072351752513496, 'movieID': 1394},
 {'rating': 4.0293191466456415, 'movieID': 51255},
 {'rating': 4.019093331734579, 'movieID': 6807},
 {'rating': 4.007109814720678, 'movieID': 166528},
 {'rating': 4.001683058605893, 'movieID': 122882},
 {'rating': 4.0, 'movieID': 260},
 {'rating': 4.0, 'movieID': 593},
 {'rating': 4.0, 'movieID': 750},
 {'rating': 4.0, 'movieID': 1036},
 {'rating': 4.0, 'movieID': 1136},
 {'rating': 4.0, 'movieID': 1210},
 {'rating': 4.0, 'movieID': 1214},
 {'rating': 4.0,

In [34]:
refinedMyAlgo.calc_similarity_matrix()

In [35]:
references = group_pref_dict[0:10]
# references = group_pref_dict
print(references)

[{'rating': 4.33073344799974, 'movieID': 318}, {'rating': 4.2775426193631025, 'movieID': 904}, {'rating': 4.235059436881213, 'movieID': 6350}, {'rating': 4.230236874437601, 'movieID': 5618}, {'rating': 4.216759592604618, 'movieID': 38061}, {'rating': 4.194045376892017, 'movieID': 60069}, {'rating': 4.130634843518455, 'movieID': 31658}, {'rating': 4.100544973206452, 'movieID': 3000}, {'rating': 4.076856035267363, 'movieID': 4011}, {'rating': 4.072612700965247, 'movieID': 2160}]


In [36]:
# references=[904, 318, 38061, 31658, 57669]
recs = refinedMyAlgo.get_similar_movies(references)

In [37]:
final_list = refinedMyAlgo.get_relevance_score(recs=recs, references=references)

for item in final_list[0:20]:
    print('Recs_dict: relevance: {}, title:{}'.format(item['movie_relevance'], item['movie_title']))

Recs_dict: relevance: 0.5353157677300935, title:Baby, The
Recs_dict: relevance: 0.48843370869678315, title:The Raid: Redemption
Recs_dict: relevance: 0.433073344799974, title:Casino
Recs_dict: relevance: 0.433073344799974, title:Shanghai Triad (Yao a yao yao dao waipo qiao)
Recs_dict: relevance: 0.433073344799974, title:Hate (Haine, La)
Recs_dict: relevance: 0.433073344799974, title:Young Poisoner's Handbook, The
Recs_dict: relevance: 0.433073344799974, title:Heavenly Creatures
Recs_dict: relevance: 0.433073344799974, title:New Jersey Drive
Recs_dict: relevance: 0.433073344799974, title:Once Were Warriors
Recs_dict: relevance: 0.433073344799974, title:Jason's Lyric
Recs_dict: relevance: 0.433073344799974, title:Above the Rim
Recs_dict: relevance: 0.4277542619363103, title:Just Cause
Recs_dict: relevance: 0.4277542619363103, title:Underneath
Recs_dict: relevance: 0.4277542619363103, title:True Crime
Recs_dict: relevance: 0.4277542619363103, title:Absolute Power
Recs_dict: relevance: 0.4

In [47]:
dist_i_R = refinedMyAlgo.calc_distance_item_in_list(item=final_list[24],this_list=final_list[0:5])
dist_i_R

0.8249255642410305